In [1]:
from ib_insync import *
import pandas as pd
from datetime import datetime, timedelta
import requests
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Dropout, LSTM, MultiHeadAttention, LayerNormalization, Add

In [2]:
util.startLoop()

In [3]:
ib = IB()
ib.connect('127.0.0.1', 7497, clientId=1)

<IB connected to 127.0.0.1:7497 clientId=1>

In [4]:
# Definir el contrato de GBP/USD
contract = Forex('GBPUSD')

# Establecer el rango de tiempo
end_time = datetime.now()
start_time = end_time - timedelta(days=30)

# Obtener datos históricos
bars = ib.reqHistoricalData(
    contract,
    endDateTime=end_time,
    durationStr='30 D',
    barSizeSetting='1 hour',
    whatToShow='MIDPOINT',
    useRTH=False
)

# Convertir los datos a DataFrame
df = util.df(bars)

In [5]:
# Calcular indicadores técnicos
df['SMA_20'] = df['close'].rolling(window=20).mean()
df['SMA_50'] = df['close'].rolling(window=50).mean()
df['RSI'] = 100 - (100 / (1 + (df['close'].diff().clip(lower=0).rolling(window=14).mean() /
                               df['close'].diff().clip(upper=0).abs().rolling(window=14).mean())))
macd_fast = df['close'].ewm(span=12, adjust=False).mean()
macd_slow = df['close'].ewm(span=26, adjust=False).mean()
df['MACD'] = macd_fast - macd_slow

# Eliminar valores NaN
df.dropna(inplace=True)

In [6]:
# Clave de API de News API
API_KEY = 'b35c56d955ee45178c703f7f79c1dfca'
news_url = f'https://newsapi.org/v2/everything?q=GBP%20USD&from={end_time - timedelta(hours=24)}&apiKey={API_KEY}'
response = requests.get(news_url)
news_data = response.json()


In [7]:
# Extraer titulares relevantes
titles = [article['title'] for article in news_data['articles']]
print(f"Titulares de noticias relevantes:\n {titles}")

Titulares de noticias relevantes:
 ['Dollar slips lower ahead of CPI; euro awaits ECB meeting']


In [8]:
# Contar palabras clave positivas y negativas
positive_keywords = ['increase', 'growth', 'positive']
negative_keywords = ['decline', 'decrease', 'negative']

positive_news = sum(1 for title in titles if any(word in title.lower() for word in positive_keywords))
negative_news = sum(1 for title in titles if any(word in title.lower() for word in negative_keywords))

In [9]:
# Impacto total en los precios
impact_factor = 1 + (positive_news - negative_news) * 0.001

In [10]:
# Seleccionar las columnas relevantes y normalizar los datos
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df[['open', 'high', 'low', 'close', 'SMA_20', 'SMA_50', 'RSI', 'MACD']].values)
scaled_data_adjusted = scaled_data * impact_factor

In [11]:
# Generar datos sintéticos con simulación de Monte Carlo
monte_carlo_data = []
num_simulations = 10
for _ in range(num_simulations):
    noise = np.random.normal(0, 0.01, scaled_data.shape)
    monte_carlo_data.append(scaled_data + noise)

scaled_data_adjusted = np.vstack([scaled_data_adjusted] + monte_carlo_data)

In [12]:
# Crear datos de prueba simulados para el ejemplo (reemplaza con tus datos reales)
# Este es el tamaño de tus datos ajustados con noticias, Monte Carlo, etc.
data_length = 1000  # Cantidad de datos de ejemplo
scaled_data_adjusted = np.random.rand(data_length, 4)  # Simular datos ajustados con solo 4 columnas relevantes

# Parámetros
sequence_length = 60  # Longitud de la secuencia para el modelo
num_features = 4  # Número de características: open, high, low, close

# Crear las secuencias para entrenamiento
x_train, y_train = [], []
for i in range(sequence_length, len(scaled_data_adjusted) - sequence_length):
    # Crear una secuencia de longitud 60 para x_train
    x_train.append(scaled_data_adjusted[i - sequence_length:i])  # Secuencia de longitud 60
    # Crear la salida correspondiente, la secuencia posterior de longitud 60
    y_train.append(scaled_data_adjusted[i:i + sequence_length])  # Predecir la siguiente secuencia de longitud 60

x_train = np.array(x_train)
y_train = np.array(y_train)

# Verificar las formas
print(f"x_train shape: {x_train.shape}")  # Debería ser (num_muestras, 60, 4)
print(f"y_train shape: {y_train.shape}")  # Debería ser (num_muestras, 60, 4)

x_train shape: (880, 60, 4)
y_train shape: (880, 60, 4)


In [24]:
# Crear la arquitectura del modelo Transformer
input_layer = Input(shape=(sequence_length, num_features))  # Ahora el input es solo 4 columnas con secuencias de longitud 60

def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Capa de atención multi-cabezal
    attention = MultiHeadAttention(num_heads=num_heads, key_dim=head_size)(inputs, inputs)
    attention = Dropout(dropout)(attention)
    attention = Add()([attention, inputs])
    attention = LayerNormalization(epsilon=1e-6)(attention)

    # Red de avance (Feed Forward Network)
    ff = Dense(ff_dim, activation="relu")(attention)
    ff = Dropout(dropout)(ff)
    ff = Dense(inputs.shape[-1])(ff)
    ff = Add()([ff, attention])
    return LayerNormalization(epsilon=1e-6)(ff)

# Aplicar varias capas de transformer
x = transformer_encoder(input_layer, head_size=64, num_heads=4, ff_dim=128, dropout=0.1)
x = transformer_encoder(x, head_size=64, num_heads=4, ff_dim=128, dropout=0.1)

# Agregar capas densas
x = Dense(64, activation="relu")(x)
x = Dropout(0.1)(x)
output_layer = Dense(num_features)(x)  # Predicción de la próxima secuencia

# Crear el modelo
model = Model(inputs=input_layer, outputs=output_layer)

# Compilar el modelo
model.compile(optimizer='adam', loss='mean_squared_error')

# Resumen del modelo
model.summary()

# Entrenar el modelo
model.fit(x_train, y_train, epochs=20, batch_size=32)

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 60, 4)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 60, 4)     │      4,868 │ input_layer_1[0]… │
│ (MultiHeadAttentio… │                   │            │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_8 (Dropout) │ (None, 60, 4)     │          0 │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_4 (Add)         │ (None, 60, 4)     │          0 │ dropout_8[0][0],  │
│                     │                   │            │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 60, 4)     │          8 │ add_4[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 60, 128)   │        640 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_9 (Dropout) │ (None, 60, 128)   │          0 │ dense_6[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, 60, 4)     │        516 │ dropout_9[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_5 (Add)         │ (None, 60, 4)     │          0 │ dense_7[0][0],    │
│                     │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 60, 4)     │          8 │ add_5[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 60, 4)     │      4,868 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_11          │ (None, 60, 4)     │          0 │ multi_head_atten… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_6 (Add)         │ (None, 60, 4)     │          0 │ dropout_11[0][0], │
│                     │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 60, 4)     │          8 │ add_6[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_8 (Dense)     │ (None, 60, 128)   │        640 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_12          │ (None, 60, 128)   │          0 │ dense_8[0][0]     │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_9 (Dense)     │ (None, 60, 4)     │        516 │ dropout_12[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_7 (Add)         │ (None, 60, 4)     │          0 │ dense_9[0][0],    │
│                     │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 60, 4)     │          8 │ add_7[0][0]     

 Total params: 12,660 (49.45 KB)

 Trainable params: 12,660 (49.45 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 6s 26ms/step - loss: 0.2526
Epoch 2/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 0.0971
Epoch 3/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 0.0935
Epoch 4/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 0.0918
Epoch 5/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 0.0900
Epoch 6/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 0.0891
Epoch 7/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - loss: 0.0877
Epoch 8/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 0.0875
Epoch 9/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - loss: 0.0870
Epoch 10/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - loss: 0.0867
Epoch 11/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - loss: 0.0862
Epoch 12/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - loss: 0.0861
Epoch 13/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 0.0855
Epoch 14/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 0.0855
Epoch 15/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 0.0851
Epoc